# eXamine Project



1. The Dataset is in a folder named "Dataset_clean" with a total of 780 ultrasound images acquired from public databases. These images are further split into train and validation sets, each containing benign, malignant, and normal ultrasound images. These three classes will be the final output of our machine learning model.


2. A total of three models have been tested and trained to find the best model with the highest validation accuracy: neural network, transfer learning with Resnet50, and convolutional neural network. Considering runtime, validation accuracy, and validation loss, the best model was found to be a convolutional neural network model trained from scratch.

In [ ]:
train_dir = 'datax/Dataset_clean/Datasets'

# 1. Neural Network

In [ ]:
pip install opencv-python # install this if you don't have opencv already

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
path = os.path.join(os.path.expanduser('~'), 'datax/Dataset_clean/Datasets')

In [ ]:
import cv2
DATADIR = path
CATEGORIES = ["benign", "malignant", "normal"]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)  
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display

        break
    break 

In [ ]:
print(img_array.shape)

## Resizing images

In [ ]:
IMG_SIZE = 100

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show()

## Create training data

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  # create path
        class_num = CATEGORIES.index(category)  # get the classification  (0, 1, 2)

        for img in os.listdir(path):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # keeping the output clean...
                pass

In [ ]:
create_training_data()

print(len(training_data))

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

## Storing X and y in pickle

In [ ]:
# SLIDES

import pickle #save this data

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
from tensorflow import keras
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras.backend import clear_session
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np

In [ ]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# SLIDES

X_train_sample, y_train_sample = resample(X,y, replace=False,n_samples=1578, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_sample,y_train_sample,test_size=0.2,
                                                  stratify=y_train_sample)

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [ ]:
# SLIDES

print("Shape of the training images array:", X_train.shape)
print("Shape of the training labels array:", y_train.shape)
print("Shape of the validation images array:",X_val.shape)
print("Shape of the validation labels array:",y_val.shape)

In [ ]:
X_train = np.array(X_train) / 255
X_val   = np.array(X_val) / 255

In [ ]:
# SLIDES

fig = plt.figure(constrained_layout=True,figsize=(15,4.5))
spec_arr = gridspec.GridSpec(ncols=10, nrows=3, figure=fig)
for ximg, ylabel, spec in zip(X_train,y_train,spec_arr):
  ax = fig.add_subplot(spec)
  ax.imshow(ximg)
  ax.set_title("({0})".format(ylabel))
  ax.set_xticks([])
  ax.set_yticks([])

## Create our neural network model

In [ ]:
# SLIDES

clear_session()
initializer_G = keras.initializers.GlorotNormal(seed=0)

model_basic = keras.Sequential([
    keras.layers.Flatten(input_shape=[100,100]),
    keras.layers.Dense(units=100,activation="sigmoid",kernel_initializer=initializer_G),
    keras.layers.Dense(units=3,activation="softmax",kernel_initializer=initializer_G)])

# model_basic.compile(loss="sparse_categorical_crossentropy",
#               optimizer=keras.optimizers.SGD(learning_rate=1),
#               metrics=["accuracy"])

model_basic.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.005),
              metrics=["accuracy"])

print(model_basic.summary())

history = model_basic.fit(X_train,y_train,validation_data=(X_val,y_val),batch_size=10,epochs=50,shuffle=False,verbose=1)
lr_num = keras.backend.eval(history.model.optimizer.learning_rate)
fig, AX = plt.subplots(nrows=1,ncols=2,figsize=(9,4))
# plot_learning(history_basic,'lr=1.0','C0',AX[0],AX[1])
plot_learning(history,'lr={0}'.format(lr_num),'C1',AX[0],AX[1])

# 2. Transfer Learning

In [ ]:
import numpy as np
import os
import time
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
from keras.applications.resnet import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
img_path = '/home/jovyan/datax/Datasets/Datasets_CNN/Training/benign/benign (51).png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
print (x.shape)
x = np.expand_dims(x, axis=0)
print (x.shape)
x = preprocess_input(x)
print('Input image shape:', x.shape)

In [ ]:
# Loading the training data
DATADIR = "Datasets"
CATEGORIES = ["benign", "malignant", "normal"]

In [ ]:
PATH = '/home/jovyan/datax/Datasets/Datasets_CNN'
# Define data path

data_dir_list = os.listdir(PATH)
data_dir_list

In [ ]:
img_data_list=[]

for dataset in data_dir_list:
	img_list=os.listdir(PATH+'/'+ dataset) 
	for img in img_list:
		img_path = PATH + '/'+ dataset + '/'+ img 
		img = image.load_img(img_path, target_size=(224, 224))
		#a = img.rotate(90)
		#b = img.rotate(180)
		#c = img.rotate(270)
		x = image.img_to_array(img)
		#a1 = image.img_to_array(a)
		#b1 = image.img_to_array(b)
		#c1 = image.img_to_array(c)
		x = np.expand_dims(x, axis=0)
		#a1 = np.expand_dims(a1, axis=0)
		#b1 = np.expand_dims(b1, axis=0)
		#c1 = np.expand_dims(c1, axis=0)
		x = preprocess_input(x)
		#a1 = preprocess_input(a1)
		#b1 = preprocess_input(b1)
		#c1 = preprocess_input(c1)
		#print('Input image shape:', x.shape)
		#print('Input image shape:', a1.shape)
		#print('Input image shape:', b1.shape)
		#print('Input image shape:', c1.shape)
		img_data_list.append(x)
		#img_data_list.append(a1)
		#img_data_list.append(b1)
		#img_data_list.append(c1)

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)
print(len(img_data_list))

In [ ]:
len(img_data)

In [ ]:
# Define the number of classes
num_classes = 3
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:260]=0
labels[260:520]=1
labels[520:]=2

In [ ]:
names = ["benign", "malignant", "normal"]
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

In [ ]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
image_input = Input(shape=(224, 224, 3))
model = ResNet50(weights='imagenet',include_top=False)
model.summary()
last_layer = model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(last_layer)
# add fully-connected & dropout layers
x = Dense(512, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu',name='fc-2')(x)
x = Dropout(0.5)(x)
# a softmax layer for 4 classes
out = Dense(num_classes, activation='softmax',name='output_layer')(x)

# this is the model we will train
custom_resnet_model2 = Model(inputs=model.input, outputs=out)

custom_resnet_model2.summary()

for layer in custom_resnet_model2.layers[:-6]:
	layer.trainable = False

custom_resnet_model2.layers[-1].trainable

custom_resnet_model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
hist = custom_resnet_model2.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data=(X_val, y_val))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_resnet_model2.evaluate(X_val, y_val, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

In [ ]:
path = 'Mendeley Database'
# Define data path

new_data_dir_list = os.listdir(path)
new_data_dir_list

# Convolutional Neural Network

In [ ]:
import os

In [ ]:
base_dir = '/home/jovyan/datax/Datasets/Datasets_CNN'
train_dir = os.path.join(base_dir, 'Training')
validation_dir = os.path.join(base_dir, 'Validation')

train_b_dir = os.path.join(train_dir, 'benign')
train_m_dir = os.path.join(train_dir, 'malignant')
train_n_dir = os.path.join(train_dir, 'normal')

validation_b_dir = os.path.join(validation_dir, 'benign')
validation_m_dir = os.path.join(validation_dir, 'malignant')
validation_n_dir = os.path.join(validation_dir, 'normal')

In [ ]:
train_b_fnames = os.listdir(train_b_dir)
#print(train_b_fnames[:10])

train_m_fnames = os.listdir(train_m_dir)
#print(train_m_fnames[:10])

train_n_fnames = os.listdir(train_n_dir)

In [ ]:
print('total training benign images:', len(os.listdir(train_b_dir)))
print('total training malignant images:', len(os.listdir(train_m_dir)))
print('total training normal images:', len(os.listdir(train_n_dir)))
print('total validation benign images:', len(os.listdir(validation_b_dir)))
print('total validation malignant images:', len(os.listdir(validation_m_dir)))
print('total validation normal images:', len(os.listdir(validation_n_dir)))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

img_path = os.path.join(train_b_dir, train_b_fnames[2])
img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# The .flow() command below generates batches of randomly transformed images
i = 0
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))
for a in range(3):
  for batch in datagen.flow(x, batch_size=1):
    image = array_to_img(batch[0])
    ax[a].imshow(image)
    ax[a].axis('off')
    #plt.figure(i)
    #imgplot = plt.imshow(img)
    i += 1
    if i % 5 == 0:
      break

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.2,1.0],
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150, 150), 
                                                    batch_size=20, class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(150, 150), 
                                                       batch_size=20, class_mode='categorical')

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator.class_indices

## Residual Module

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import label_binarize

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import add
from keras.utils import plot_model
 
# function for creating an identity or projection residual module
def residual_module(layer_in, n_filters):
	merge_input = layer_in
	# check if the number of filters needs to be increase, assumes channels last format
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv1
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv2
	conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	# add filters, assumes filters/channels last
	layer_out = add([conv2, merge_input])
	# activation function
	layer_out = Activation('relu')(layer_out)
	return layer_out

## Creating the model

In [ ]:
img_input = layers.Input(shape=(150, 150, 3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Convolution2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

#x = residual_module(x, 64)

x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dropout(0.5)(x)

output = layers.Dense(3, activation='softmax')(x)

model = Model(img_input, output)
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])

In [ ]:
history = model.fit(train_generator, 
                    steps_per_epoch=int(701/20), epochs=20, validation_data=validation_generator, 
                    validation_steps=int(78/20), verbose=1)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label = 'training')
plt.plot(epochs, val_acc, label = 'validation')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, label = 'training')
plt.plot(epochs, val_loss, label = 'validation')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
y_pred = model.predict(validation_generator)
y_test = validation_generator.classes
y_test = label_binarize(y_test, classes=[0, 1, 2])

## ROC-AUC

In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

n_classes = 3
# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for 3 classes of abnormalities')
plt.legend(loc="lower right")
plt.show()

Feature Extraction Using ResNet50

In [ ]:
path = 'Dataset_clean/train/normal/normal (13).png'
img_test = load_img(path, target_size=(150, 150))
print(img_test)
img_test = img_to_array(img_test)
img_test = img_test.reshape((1,) + img_test.shape)
img_test.shape

In [ ]:
model.predict(img_test)

In [ ]:
base_dir = 'Mendeley Database'
test_dir = os.path.join(base_dir, 'test')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,target_size=(150, 150), batch_size=20, class_mode='categorical')
model.evaluate(test_generator)

## Saving model to pickle

In [ ]:
import pickle
filename = 'model_weights_430.pickle'
outfile = open(filename,'wb')

pickle.dump(w_old,outfile)
outfile.close()

In [ ]:
infile = open(filename,'rb')
w_old_unpickled = pickle.load(infile)
infile.close()

len(w_old_unpickled) #weights

In [ ]:
w_old_unpickled[1]

In [ ]:
w_old[1]

In [ ]:
# this should show the confidence of the model on each diagnosis (my terminology might be off)



pred1 = model1.predict(img_test)[0]
tot = sum(pred1)
print([i/tot for i in pred1])

# print(pred1)

print(model.predict(img_test)[0])

In [ ]:
model1.layers[10].activation
model.layers[10].activation

pred1

In [ ]:
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link
import ipywidgets as widgets